In [1]:
import csv
import numpy as np
import tensorflow as tf
import pandas as pd
import re
import nltk
from langdetect import detect

## Load the dataset

In [ ]:
# open the file in read mode
with open('ArCovidVac.txt', encoding='utf-8') as file:
    lines = file.readlines()

In [6]:
#convert to csv file

with open('ArCovidVac.txt', 'r', encoding='utf-8-sig') as txtfile, open('AraCovidVac.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    txt_reader = csv.reader(txtfile, delimiter='\t')
    csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for row in txt_reader:
        csv_writer.writerow(row)

In [ ]:
# load the dataset
data = pd.read_csv('synArbP1.csv')
data.head()

In [ ]:
data['text'].head(10) , data['stance'].value_counts()

## preprocessing

In [3]:
# remove hashtag
def removeHT(text):
    tweet = re.sub(r'#\S+', '', text)
    return tweet

data['Translated_Text']= data['Translated_Text'].apply(lambda x: removeHT(x))

In [ ]:
def removeURLs(text):
    tweet = re.sub(r'http\S+',' ', text)
    tweet = re.sub('_',' ', text)
    return tweet

data['Translated_Text']= data['Translated_Text'].apply(lambda x: removeURLs(x))
data['Translated_Text'] = data['Translated_Text'].apply(lambda x: removeURLs(x))

data.head()

In [5]:

def remove_identical_words(sentence):
    # Tokenize the sentence into words
    words = re.findall(r'\b\w+\b', sentence)

    # Remove consecutive identical words
    filtered_words = [words[0]]  # Initialize with the first word
    for i in range(1, len(words)):
        if words[i] != words[i - 1]:
            filtered_words.append(words[i])

    # Recreate the sentence without consecutive identical words
    result = ' '.join(filtered_words)

    return result

data['Translated_Text'] = data['Translated_Text'].apply(lambda x: remove_identical_words(x))

In [ ]:
# text cleaning
#remove non word characters, &amp, new line indicators, convert to same case and translate
def clean_text(tweet):
    tweet = re.sub(r'&\S+', ' ', tweet) # remove '&amp'
    tweet = tweet.replace("\\n",' ') # remove end of line signs '\n'
    tweet = re.sub(r'[^\w\s]',' ',tweet) # remove non word characters
    tweet = re.sub(r'@\w*', " ", tweet) # remove usernames
    tweet = re.sub(r'ههه+','', tweet) # remove هههههههههههههههههه
    tweet = tweet.lower() #convert to lower case
    #tweet = re.sub(r'[0-9]','',tweet) #remove numbers
    tweet = re.sub(r'[a-zA-Z]', ' ', tweet)
    emojis = re.compile("["
                    u"\U0001F600-\U0001F64F"
                    u"\U0001F300-\U0001F5FF"
                    u"\U0001F680-\U0001F6FF"
                    u"\U0001F1E0-\U0001F1FF"
                    u"\U00002702-\U000027B0"
                    u"\U000024C2-\U0001F251"
                    "]+", flags=re.UNICODE)
    tweet = emojis.sub(r'', tweet) if emojis.search(tweet) else tweet # remove emojis
    return tweet



entry = data.iloc[0]['Translated_Text']
print('raw data sample: ', entry)
print('after cleaning: ', clean_text(entry))

data['clean_Translated_Text'] = data['Translated_Text'].apply(lambda x: clean_text(x))
data['clean_Translated_Text'] = data['Translated_Text'].apply(lambda x: clean_text(x))
data.head()

In [ ]:
#Removing White Spaces
data['clean_Translated_Text'] = data['clean_Translated_Text'].apply(lambda text: " ".join(text.split()))
data.head(10)

In [ ]:
data['cleanText'][2]

In [11]:
train_data = pd.concat([data['clean_Translated_Text'],data['Translated_Text']], axis=1)
train_data.to_csv('synArbP1_prepro.csv', index=False)

In [60]:
# Sample dataset
dataset = ["مرحبًا بك في قطر", "Hello, this is an English text.", "معمول في قطر"]

# Function to detect language and filter out Qatari dialect
def is_qatari_dialect(text):
    try:
        language = detect(text)
        # Adjust the language code for Qatari Arabic as needed
        return language != 'ar'
    except:
        return False

# Filter out Qatari dialect texts
non_qatari_texts = [text for text in dataset if not is_qatari_dialect(text)]

# Print the filtered texts
for text in non_qatari_texts:
    print(text)


مرحبًا بك في قطر
معمول في قطر
